<a href="https://colab.research.google.com/github/baohl00/OphNER/blob/main/%5BOphNER%5D_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 117.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 129.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 120.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
np.random.seed(0)
plt.style.use("ggplot")
import tensorflow as tf
from simpletransformers.ner import NERModel, NERArgs

print('Tensorflow version:', tf.__version__)
print('GPU detected:', tf.config.list_physical_devices('GPU'))

Tensorflow version: 2.12.0
GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
train = pd.read_csv('train1.csv', encoding= 'unicode_escape')
test = pd.read_csv('test1.csv', encoding= 'unicode_escape')

train.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
test.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# Convert training set and dev set to Simpletransformer format
train = pd.DataFrame({'sentence_id': train['sentence_id'].values,
                      'words': train['word'].astype(str).values,
                      'labels': train['label'].astype(str).values})
test = pd.DataFrame({'sentence_id': test['sentence_id'].values,
                     'words': test['word'].astype(str).values,
                     'labels': test['label'].astype(str).values})

In [ ]:
# Get number of tag in training dataset
tags = list(set(list(train["labels"].values)+list(test['labels'].values)))
num_tags = len(tags)

In [ ]:
num_tags

33

In [ ]:
tags

['B-SIGN_AND_SYMPTOM',
 'B-NAME_OF_STUDY',
 'B-NAME_OF_DRUG',
 'B-GPE',
 'B-CAUSE_OF_DISEASE',
 'I-PART_OF_THE_EYES',
 'B-PUBLICATION_VENUE',
 'I-NAME_OF_DRUG',
 'B-SURGICAL_INSTRUMENT',
 'B-TIME_DURATION',
 'I-ORGANIZATION',
 'B-NAME_OF_DISEASE',
 'I-TIME_DURATION',
 'I-NAME_OF_STUDY',
 'I-PERSON',
 'B-PERSON',
 'I-PUBLICATION_VENUE',
 'I-NAME_OF_DISEASE',
 'B-DIAGNOSTIC_TEST',
 'B-PREVENTION',
 'O',
 'I-GPE',
 'I-SIGN_AND_SYMPTOM',
 'B-CHEMICAL_COMPOUND',
 'I-SURGICAL_INSTRUMENT',
 'I-PREVENTION',
 'B-ORGANIZATION',
 'B-TREATMENT',
 'I-CAUSE_OF_DISEASE',
 'B-PART_OF_THE_EYES',
 'I-TREATMENT',
 'I-CHEMICAL_COMPOUND',
 'I-DIAGNOSTIC_TEST']

In [ ]:
len(tags)

33

In [ ]:
len(set(list(train['sentence_id']))), len(set(list(test['sentence_id'])))

(8156, 784)

In [ ]:
train_label = train['labels'].value_counts()
test_label = test['labels'].value_counts()
pd.DataFrame(pd.concat([train_label, test_label], axis=1))

,labels,labels
O,126057,13540
I-SIGN_AND_SYMPTOM,7498,186
B-NAME_OF_DISEASE,5097,353
I-TREATMENT,4890,336
I-CAUSE_OF_DISEASE,3732,52
I-NAME_OF_DISEASE,3670,226
B-TREATMENT,2509,278
B-SIGN_AND_SYMPTOM,1676,63
I-PREVENTION,1566,92
I-PART_OF_THE_EYES,1501,195


In [ ]:

# # Set up param for model

# args = NERArgs()
# args.num_train_epochs = 10
# args.learning_rate = 2e-5
# args.overwrite_output_dir =True
# args.train_batch_size = 10
# args.eval_batch_size = 10
# args.use_cached_eval_features = False
# args.use_multiprocessing = False
# args.reprocess_input_data = True
# model = NERModel("auto", "bert-base-uncased", labels=tags, args=args)
# model.train_model(train, validation_data=test, acc=accuracy_score)
# result, model_outputs, wrong_predictions = model.eval_model(test)
# result

# model func

In [ ]:
import time

result_df = pd.DataFrame({'eval_loss':[], 'precision':[], 'recall':[],'f1_score':[], 'name_model':[], 'time': []})

def model(key, name_model):
    # Set up param for model

    args = NERArgs()
    args.num_train_epochs = 10
    args.learning_rate = 2e-5
    args.overwrite_output_dir =True
    args.train_batch_size = 10
    args.eval_batch_size = 10
    args.use_cached_eval_features = False
    args.use_multiprocessing = False
    args.reprocess_input_data = True

    model = NERModel(key, name_model, labels=tags, args=args)
    start = time.time()
    model.train_model(train, validation_data=test, acc=accuracy_score)
    result, model_outputs, wrong_predictions = model.eval_model(test)
    end = time.time()
    predictions, raw_outputs = model.predict(["Millions of people worldwide suffer from the itching burning and other discomforts of dry eye disease DED",
                                              "Autologous serum contains proteins lysozyme lactoferrin albumin and immunoglobulins growth factors epidermal growth factor vitamins A and C antioxidants and electrolytes sodium potassium chloride bicarbonate magnesium calcium",
                                              "The i-ROP system is capable of categorizing fundus photos into type 1 type 2 and pre-plus ROP with probability scores of 0.96 and 0.91 for detecting type 1 ROP and clinically significant ROP respectively"])
    result['name_model'] = name_model
    result['time'] = end - start

    # result_df = result_df.append(result, ignore_index=True)
    return result, predictions

In [ ]:
  # "label": [
  #           {
  #               "start": 41,
  #               "end": 82,
  #               "text": "the itching burning and other discomforts",
  #               "labels": [
  #                   "SIGN_AND_SYMPTOM"
  #               ]
  #           },
  #           {
  #               "start": 86,
  #               "end": 105,
  #               "text": "dry eye disease DED",
  #               "labels": [
  #                   "NAME_OF_DISEASE"
  #               ]
  #           }
  #       ]

In [ ]:
tt_df = pd.DataFrame({'name':[], 'eval_loss':[], 'precision':[], 'recall':[],'f1_score':[]})

display(tt_df)
t1 = {'name': 'a',
 'eval_loss': 0.29787937791346086,
  'precision': 0.6468646864686468,
  'recall': 0.698005698005698,
  'f1_score': 0.671462829736211}
tt_df = tt_df.append(t1, ignore_index=True)
display(tt_df)

,name,eval_loss,precision,recall,f1_score


<ipython-input-12-349d09efeb38>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tt_df = tt_df.append(t1, ignore_index=True)


,name,eval_loss,precision,recall,f1_score
0,a,0.297879,0.646865,0.698006,0.671463


In [ ]:
temp = {'name': ['a'],
  'eval_loss': [0.29787937791346086],
  'precision': [0.6468646864686468],
  'recall': [0.698005698005698],
  'f1_score': [0.671462829736211]}

pd.DataFrame(temp)

,name,eval_loss,precision,recall,f1_score
0,a,0.297879,0.646865,0.698006,0.671463


## xlm-roberta-base

In [ ]:
result_model = model("xlmroberta", "xlm-roberta-base")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-14-42a2d8f25a28>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


## xlm-roberta-large

In [ ]:
result_model = model("xlmroberta", "xlm-roberta-large")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-12-22702effe1a5>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


In [ ]:
result_model[1]

[[{'Millions': 'O'},
  {'of': 'O'},
  {'people': 'O'},
  {'worldwide': 'O'},
  {'suffer': 'O'},
  {'from': 'O'},
  {'the': 'B-SIGN_AND_SYMPTOM'},
  {'itching': 'I-SIGN_AND_SYMPTOM'},
  {'burning': 'I-SIGN_AND_SYMPTOM'},
  {'and': 'I-SIGN_AND_SYMPTOM'},
  {'other': 'I-SIGN_AND_SYMPTOM'},
  {'discomforts': 'I-SIGN_AND_SYMPTOM'},
  {'of': 'O'},
  {'dry': 'B-NAME_OF_DISEASE'},
  {'eye': 'I-NAME_OF_DISEASE'},
  {'disease': 'I-NAME_OF_DISEASE'},
  {'DED': 'I-NAME_OF_DISEASE'}],
 [{'Autologous': 'B-NAME_OF_DRUG'},
  {'serum': 'I-NAME_OF_DRUG'},
  {'contains': 'O'},
  {'proteins': 'B-CHEMICAL_COMPOUND'},
  {'lysozyme': 'B-CHEMICAL_COMPOUND'},
  {'lactoferrin': 'B-CHEMICAL_COMPOUND'},
  {'albumin': 'B-CHEMICAL_COMPOUND'},
  {'and': 'O'},
  {'immunoglobulins': 'B-CHEMICAL_COMPOUND'},
  {'growth': 'O'},
  {'factors': 'O'},
  {'epidermal': 'O'},
  {'growth': 'O'},
  {'factor': 'O'},
  {'vitamins': 'B-CHEMICAL_COMPOUND'},
  {'A': 'I-CHEMICAL_COMPOUND'},
  {'and': 'I-CHEMICAL_COMPOUND'},
  {'C': 'I-

In [ ]:
result_df

,eval_loss,precision,recall,f1_score,name_model,time
0,0.399851,0.640368,0.693732,0.665983,xlm-roberta-base,695.407728
1,0.446202,0.680667,0.727208,0.703168,xlm-roberta-large,1418.880720


## bert-base-uncased

In [ ]:
presult_model = model("auto", "bert-base-uncased")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-17-06fb68fd40ad>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


## bert-large-uncased

In [ ]:
result_model = model("auto", "bert-large-uncased")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-18-81dff9739c3e>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


In [ ]:
result_df

,eval_loss,precision,recall,f1_score,name_model,time
0,0.399851,0.640368,0.693732,0.665983,xlm-roberta-base,695.407728
1,0.446202,0.680667,0.727208,0.703168,xlm-roberta-large,1418.880720
2,0.507308,0.525575,0.585470,0.553908,bert-base-uncased,515.575099
3,0.559768,0.571146,0.617521,0.593429,bert-large-uncased,1126.260649


## roberta-base

In [ ]:
result_model = model("roberta", "roberta-base")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-20-75ddb6712b52>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


## roberta-large

In [ ]:
result_model = model("roberta", "roberta-large")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be ab

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-12-ab219b1e5fd5>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


In [ ]:
result_model[1]

[[{'Millions': 'O'},
  {'of': 'O'},
  {'people': 'O'},
  {'worldwide': 'O'},
  {'suffer': 'O'},
  {'from': 'O'},
  {'the': 'B-SIGN_AND_SYMPTOM'},
  {'itching': 'I-SIGN_AND_SYMPTOM'},
  {'burning': 'I-SIGN_AND_SYMPTOM'},
  {'and': 'I-SIGN_AND_SYMPTOM'},
  {'other': 'I-SIGN_AND_SYMPTOM'},
  {'discomforts': 'I-SIGN_AND_SYMPTOM'},
  {'of': 'O'},
  {'dry': 'B-NAME_OF_DISEASE'},
  {'eye': 'I-NAME_OF_DISEASE'},
  {'disease': 'I-NAME_OF_DISEASE'},
  {'DED': 'I-NAME_OF_DISEASE'}],
 [{'Autologous': 'B-NAME_OF_DRUG'},
  {'serum': 'I-NAME_OF_DRUG'},
  {'contains': 'O'},
  {'proteins': 'B-CHEMICAL_COMPOUND'},
  {'lysozyme': 'B-CHEMICAL_COMPOUND'},
  {'lactoferrin': 'B-CHEMICAL_COMPOUND'},
  {'albumin': 'B-CHEMICAL_COMPOUND'},
  {'and': 'O'},
  {'immunoglobulins': 'B-CHEMICAL_COMPOUND'},
  {'growth': 'O'},
  {'factors': 'O'},
  {'epidermal': 'O'},
  {'growth': 'O'},
  {'factor': 'O'},
  {'vitamins': 'B-CHEMICAL_COMPOUND'},
  {'A': 'I-CHEMICAL_COMPOUND'},
  {'and': 'I-CHEMICAL_COMPOUND'},
  {'C': 'I-

## xlnet-base-cased

In [ ]:
result_model = model("auto", "xlnet-base-cased")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForTokenClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForTokenClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-22-6ca1583fc9a4>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


## xlnet-large-cased

In [ ]:
result_model = model("auto", "xlnet-large-cased")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForTokenClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForTokenClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-14-9bf683648392>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


In [ ]:
result_model[1]

[[{'Millions': 'O'},
  {'of': 'O'},
  {'people': 'O'},
  {'worldwide': 'O'},
  {'suffer': 'O'},
  {'from': 'O'},
  {'the': 'B-SIGN_AND_SYMPTOM'},
  {'itching': 'I-SIGN_AND_SYMPTOM'},
  {'burning': 'I-SIGN_AND_SYMPTOM'},
  {'and': 'I-SIGN_AND_SYMPTOM'},
  {'other': 'I-SIGN_AND_SYMPTOM'},
  {'discomforts': 'I-SIGN_AND_SYMPTOM'},
  {'of': 'O'},
  {'dry': 'B-NAME_OF_DISEASE'},
  {'eye': 'I-NAME_OF_DISEASE'},
  {'disease': 'I-NAME_OF_DISEASE'},
  {'DED': 'I-NAME_OF_DISEASE'}],
 [{'Autologous': 'B-NAME_OF_DRUG'},
  {'serum': 'I-NAME_OF_DRUG'},
  {'contains': 'O'},
  {'proteins': 'B-CHEMICAL_COMPOUND'},
  {'lysozyme': 'B-CHEMICAL_COMPOUND'},
  {'lactoferrin': 'B-CHEMICAL_COMPOUND'},
  {'albumin': 'B-CHEMICAL_COMPOUND'},
  {'and': 'O'},
  {'immunoglobulins': 'B-CHEMICAL_COMPOUND'},
  {'growth': 'O'},
  {'factors': 'O'},
  {'epidermal': 'O'},
  {'growth': 'O'},
  {'factor': 'O'},
  {'vitamins': 'B-CHEMICAL_COMPOUND'},
  {'A': 'I-CHEMICAL_COMPOUND'},
  {'and': 'I-CHEMICAL_COMPOUND'},
  {'C': 'I-

In [ ]:
result_df

,eval_loss,precision,recall,f1_score,name_model,time
0,0.462750,0.678667,0.725071,0.701102,roberta-large,1163.844162
1,0.667359,0.649743,0.720085,0.683108,xlnet-large-cased,1446.587664


## bertweet-base

In [ ]:
result_model = model("auto", "vinai/bertweet-base")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-24-5dad3e85d41b>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


## distilbert-base-uncased

In [ ]:
result_model = model("auto", "distilbert-base-uncased")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-25-e357a6f01b4d>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


## distilroberta-base

In [ ]:
result_model = model("auto", "distilroberta-base")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream tas

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-26-31750169407c>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


In [ ]:
result_model = model("auto", "albert-large-v1")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at albert-large-v1 were not used when initializing AlbertForTokenClassification: ['predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-large-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probab

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-27-75d19d9ba727>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


In [ ]:
result_model = model("auto", "albert-large-v2")
result_df = result_df.append(result_model[0], ignore_index=True)

Some weights of the model checkpoint at albert-large-v2 were not used when initializing AlbertForTokenClassification: ['predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-large-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probab

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/8156 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/816 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-28-bf3d6a4516c3>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_model[0], ignore_index=True)


In [ ]:
result_df

,eval_loss,precision,recall,f1_score,name_model,time
0,0.399851,0.640368,0.693732,0.665983,xlm-roberta-base,695.407728
1,0.446202,0.680667,0.727208,0.703168,xlm-roberta-large,1418.880720
2,0.507308,0.525575,0.585470,0.553908,bert-base-uncased,515.575099
3,0.559768,0.571146,0.617521,0.593429,bert-large-uncased,1126.260649
4,0.374150,0.623274,0.675214,0.648205,roberta-base,537.765073
5,0.485039,0.680079,0.734330,0.706164,roberta-large,1177.418192
6,0.472708,0.610000,0.695157,0.649800,xlnet-base-cased,671.411206
7,0.606336,0.674252,0.738604,0.704963,xlnet-large-cased,1445.321101
8,0.402698,0.598345,0.669516,0.631933,vinai/bertweet-base,551.348107
9,0.464968,0.509206,0.571225,0.538436,distilbert-base-uncased,295.107119
